In [1]:
! pip install wikipedia
! pip install nltk
! pip install rake-nltk

  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11696 sha256=0947216086d555dd7565244de57aa01b292137d1f579ace79fdca53434c6f235
  Stored in directory: /root/.cache/pip/wheels/15/93/6d/5b2c68b8a64c7a7a04947b4ed6d89fb557dcc6bc27d1d7f3ba
Successfully built wikipedia
  Created wheel for rake-nltk: filename=rake_nltk-1.0.4-py2.py3-none-any.whl size=7829 sha256=de02f162084b760d8a9673a8405a8be7d3fadf1841d049d32408683565db05b5
  Stored in directory: /root/.cache/pip/wheels/7c/d9/8a/b8a9244fa89a07f288f9fe006aafc79d93fceb58496c29b606
Successfully built rake-nltk


In [2]:
import wikipedia
import random
import tqdm
import nltk
import re
import string
import threading
import multiprocessing
import os
import shutil
from nltk.corpus import stopwords

In [3]:
import numpy as np
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers.core import Dropout, Dense
from keras.layers import LSTM, Bidirectional, BatchNormalization
from keras.layers import SpatialDropout1D, Flatten, Conv2D
from keras.layers.embeddings import Embedding
from keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.optimizers import Adam
from keras.regularizers import l2, l1_l2
import keras

In [4]:
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [5]:
wikipedia.set_lang("ru")

In [6]:
def wikiTexts(topic, linksNum:list = [0], first:bool=True) -> (list, list) :
    links = []
    try:
        page = wikipedia.page(topic)
    except:
        return [], []
    
    try:
      links = page.links
    except:
      return result, [topic]
    
    result = [page.content]
    if len(linksNum) == 0:
        return result, [topic]
    
    random.shuffle(links)

    iterate = links[:linksNum[0]]
    resultLinks = [*iterate]
    if first:
      iterate = tqdm.tqdm_notebook(iterate)
    
    for link in iterate:
        contents = wikiTexts(link, linksNum[1:], False)
        result += contents[0]
        resultLinks += contents[1]
    return result, resultLinks

In [7]:
def wikiThread(returnDict, topic, linksNum:list = [0]):
  returnDict[topic] = wikiTexts(topic, linksNum)

In [8]:
depthSearch = [150, 2] # 150 первых ссылок, по 2 вторичных ссылки из каждой первой ссылки
jobs = []

manager = multiprocessing.Manager()
returnDict = manager.dict()

tasks = [
         (returnDict, "животные", depthSearch),
         (returnDict, "музыка", depthSearch),
         (returnDict, "спорт", depthSearch),
         (returnDict, "литература", depthSearch),
]
for i in range(len(tasks)):
  out_list = list()
  process = multiprocessing.Process(target=wikiThread, 
                                    args=tasks[i])
  jobs.append(process)

for j in jobs:
  j.start()

for j in jobs:
  j.join()
animalTexts, animalLinks = returnDict["животные"]
musicTexts, musicLinks = returnDict["музыка"]
sportTexts, sportLinks = returnDict["спорт"]
literatureTexts, literatureLinks = returnDict["литература"]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:22: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:22: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:22: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:22: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


/usr/local/lib/python3.7/dist-packages/wikipedia/wikipedia.py:389: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file /usr/local/lib/python3.7/dist-packages/wikipedia/wikipedia.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')
/usr/local/lib/python3.7/dist-packages/wikipedia/wikipedia.py:389: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that c

In [9]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [12]:
import pandas as pd
data = pd.DataFrame()
data['texts'] = animalTexts + musicTexts + sportTexts + literatureTexts
data['y'] = [0] * len(animalTexts) + [1] * len(musicTexts) + [2] * len(sportTexts) + [3] * len(literatureTexts)

In [15]:
! pip install pymorphy2[fast]

     |████████████████████████████████| 55 kB 2.2 MB/s 
     |████████████████████████████████| 8.2 MB 8.0 MB/s 
     |████████████████████████████████| 371 kB 49.3 MB/s 
  Created wheel for DAWG: filename=DAWG-0.8.0-cp37-cp37m-linux_x86_64.whl size=851958 sha256=af5e96383bc9698f49567bb8531bbccf0b6b7f92481416a1ce9c15b60a8d65a6
  Stored in directory: /root/.cache/pip/wheels/85/51/a4/2de41ff197786537075027c27b479a38da92f50abc86634445
Successfully built DAWG


In [41]:
from typing import List
import nltk
import string
import pymorphy2
import codecs

class PrepareNew():
  def __init__(self):
    self.morph = pymorphy2.MorphAnalyzer()
    self.tokenizer = nltk.WordPunctTokenizer()
    self.stopwords = set(line.strip() for line in codecs.open('/content/drive/MyDrive/aiijc_sber/all_stopwords.txt', "r", "utf_8_sig").readlines())

  def prepare_corp(self, news_list: List[str]):
    return [self.newstext2token(news_text) for news_text in news_list]

  def newstext2token(self, news_text: str):
      tokens = self.tokenizer.tokenize(news_text.lower())
      tokens_with_no_punct = [self.morph.parse(w)[0].normal_form for w in tokens if all(c in 'йцукенгшщзхъёфывапролджэячсмитьбю' for c in w)]
      tokens_base_forms = [w for w in tokens_with_no_punct if w not in self.stopwords]
      tokens_last = [w for w in tokens_base_forms if len(w)>1]
      return tokens_last


In [43]:
tokens = []
counter = 0
volume = len(data)
for i in data.texts:
  counter += 1
  tokens.append(' '.join(PrepareNew().newstext2token(i)))
  if counter%1000 == 0:
    print("Done", counter, "from", volume)
data['tokens'] = tokens


Done 1000 from 1750


In [44]:
data.to_csv('/content/drive/MyDrive/aiijc_sber/tokens_full_cleaned.csv')

In [45]:
data.head()

,texts,y,tokens
0,Живо́тные (лат. Animalia) — традиционно (со вр...,0,живо тный лата аристотель выделять организм би...
1,Пти́цы (лат. Aves) — группа теплокровных яйцек...,0,пти цы лата группа теплокровный яйцекладущий п...
2,Союз охраны птиц России (СОПР) — общероссийска...,0,союз охрана птица россия сопра общероссийский ...
3,Крючковидные отростки рёбер — костяные выросты...,0,крючковидный отросток ребро костяной вырост ве...
4,Myxozoa (лат.) — класс или подтип паразитическ...,0,лата класс подтип паразитический беспозвоночны...


In [ ]:
# for i in animalTexts:
#   with open('/content/drive/MyDrive/aiijc_sber/animals_texts.txt', 'a') as dump:
#     to_write = i.split('\n')
#     for sub in to_write:
#       dump.write(sub.strip()+"\n")

# for i in musicTexts:
#   with open('/content/drive/MyDrive/aiijc_sber/music_texts.txt', 'a') as dump:
#     to_write = i.split('\n')
#     for sub in to_write:
#       dump.write(sub.strip()+"\n")

# for i in sportTexts:
#   with open('/content/drive/MyDrive/aiijc_sber/sport_texts.txt', 'a') as dump:
#     to_write = i.split('\n')
#     for sub in to_write:
#       dump.write(sub.strip()+"\n")

# for i in literatureTexts:
#   with open('/content/drive/MyDrive/aiijc_sber/literature_texts.txt', 'a') as dump:
#     to_write = i.split('\n')
#     for sub in to_write:
#       dump.write(sub.strip()+"\n")

In [ ]:
for i in animalTexts:
  with open()

In [ ]:
allTexts = (animalTexts, musicTexts, sportTexts, literatureTexts)
allLinks = (animalLinks, musicLinks, sportLinks, literatureLinks)
categories = ("животные", "музыка", "спорт", "литература")

In [ ]:
animalLinks[3]

'Island Press'

In [ ]:
for category, texts, links in zip(categories, allTexts, allLinks):
  print(category, ":", sum([len(text.split(' ')) for text in texts]), "слов")
  print(*links, sep=", ")

животные : 692731 слов
Контрольный номер Библиотеки Конгресса, Мезозой, Голубая сойка, Island Press, Настоящее, Молоко, Ризарии, Protozoa, Нематоды, Эвгленозои, Низшие черви, Эукариоты, Ветуликолии, Ортонектиды, Головоногие, Род (биология), Первичнополостные черви, Каменноугольный период, Национальная библиотека Франции, Царство (биология), Zootaxa, Антропология, Onychophora, Holozoa, Монофилия, Tsukubamonadida, Животные в космосе, Palpitomonas, Collodictyonidae, Гомология (биология), Анус, Gemeinsame Normdatei, Opisthokonta, Видовой эпитет, Бластула, Telonemia, Морфология (биология), Парабазалии, Food and Agriculture Organization, Пермский период, Discoba, Палеозой, Двусторонне-симметричные, Нервы, Мшанки, Животные в Библии, Liocarcinus vernalis, Гады, Xenoturbella, Коралловые полипы, Стрекающие, Rappemonads, Platycryptus undatus, CRuMs, Malawimonada, Bilateria, Tubulinea, Heterolobosea, Гетеротрофы, Таксон, Верхнее озеро, Сипункулиды, Головохоботные, Книдоспоридии, Линяющие, Виноград

In [ ]:
pip install pymorphy2[fast]

     |████████████████████████████████| 55 kB 2.9 MB/s 
     |████████████████████████████████| 8.2 MB 16.2 MB/s 
     |████████████████████████████████| 371 kB 55.9 MB/s 
  Created wheel for DAWG: filename=DAWG-0.8.0-cp37-cp37m-linux_x86_64.whl size=871181 sha256=0ef9a2f79bd914afb08ab4655059c23bdfbf19a14390cf3c648fa04343cf28e4
  Stored in directory: /root/.cache/pip/wheels/85/51/a4/2de41ff197786537075027c27b479a38da92f50abc86634445
Successfully built DAWG


In [ ]:
import pymorphy2
morph = pymorphy2.MorphAnalyzer()

In [ ]:
stopWordsRu = set(stopwords.words("russian")) 
punctuation = set(string.punctuation + "—")

def validateWord(word):
  if word in stopWordsRu or word in punctuation or not word.isalpha():
    return False
  allSymValid = False

  for sym in word:
    if not sym in punctuation:
      allSymValid = True
      break

  return allSymValid

def prepareText(text:str) -> list:
  out = []
  for word in nltk.word_tokenize(text):
    word = word.strip()
    if not validateWord(word):
      continue
    word = word.lower()
    word = morph.parse(word)[0].normal_form
    if len(word)>1:
      out.append(word)
  return out


def generateTexts(texts, windowLen) -> list:
  out = []
  for text in tqdm.tqdm_notebook(texts):
    byWords = text.split(' ')
    for window in range(len(byWords) // windowLen):
      textPrepare = " ".join(byWords[window * windowLen: (window + 1) * windowLen])

      prepared = prepareText(textPrepare)
      if windowLen - len(prepared) > 0:
        prepared += [" "] * ( windowLen - len(prepared))
      out.append(prepared)
  return out

In [ ]:
sentMaxLen = 300

In [ ]:
animalsPrepared = generateTexts(animalTexts, sentMaxLen)
musicPrepared = generateTexts(musicTexts, sentMaxLen)
sportPrepared = generateTexts(sportTexts, sentMaxLen)
literaturePrepared = generateTexts(literatureTexts, sentMaxLen)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:31: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


In [ ]:
animalsPrepared[0]

['лата',
 'animalia',
 'традиционно',
 'время',
 'аристотель',
 'выделять',
 'категория',
 'организм',
 'настоящий',
 'время',
 'рассматривать',
 'качество',
 'биологический',
 'царство',
 'животное',
 'являться',
 'основный',
 'объект',
 'изучение',
 'зоология',
 'животное',
 'относиться',
 'эукариот',
 'клетка',
 'иметься',
 'ядро',
 'классический',
 'признак',
 'животное',
 'считаться',
 'гетеротрофность',
 'питание',
 'готовый',
 'органический',
 'соединение',
 'способность',
 'активно',
 'передвигаться',
 'впрочем',
 'существовать',
 'немало',
 'животное',
 'ведущий',
 'неподвижный',
 'образ',
 'жизнь',
 'гетеротрофность',
 'свойственный',
 'гриб',
 'некоторый',
 'русский',
 'слово',
 'животное',
 'образовать',
 'живот',
 'прошлый',
 'означать',
 'жизнь',
 'имущество',
 'быт',
 'термин',
 'дикий',
 'животное',
 'домашний',
 'животное',
 'часто',
 'пониматься',
 'млекопитающее',
 'четвероногий',
 'наземный',
 'позвоночный',
 'млекопитающее',
 'пресмыкаться',
 'земноводное',
 'однак

In [ ]:
dataFullX = animalsPrepared + musicPrepared + sportPrepared + literaturePrepared
dataFullY = np.array([[1, 0, 0, 0]] * len(animalsPrepared) + [[0, 1, 0, 0]] * len(musicPrepared) + [[0, 0, 1, 0]] * len(sportPrepared) + [[0, 0, 0, 1]] * len(literaturePrepared))

In [ ]:
dataFullX[0]

['лата',
 'animalia',
 'традиционно',
 'время',
 'аристотель',
 'выделять',
 'категория',
 'организм',
 'настоящий',
 'время',
 'рассматривать',
 'качество',
 'биологический',
 'царство',
 'животное',
 'являться',
 'основный',
 'объект',
 'изучение',
 'зоология',
 'животное',
 'относиться',
 'эукариот',
 'клетка',
 'иметься',
 'ядро',
 'классический',
 'признак',
 'животное',
 'считаться',
 'гетеротрофность',
 'питание',
 'готовый',
 'органический',
 'соединение',
 'способность',
 'активно',
 'передвигаться',
 'впрочем',
 'существовать',
 'немало',
 'животное',
 'ведущий',
 'неподвижный',
 'образ',
 'жизнь',
 'гетеротрофность',
 'свойственный',
 'гриб',
 'некоторый',
 'русский',
 'слово',
 'животное',
 'образовать',
 'живот',
 'прошлый',
 'означать',
 'жизнь',
 'имущество',
 'быт',
 'термин',
 'дикий',
 'животное',
 'домашний',
 'животное',
 'часто',
 'пониматься',
 'млекопитающее',
 'четвероногий',
 'наземный',
 'позвоночный',
 'млекопитающее',
 'пресмыкаться',
 'земноводное',
 'однак

In [ ]:
len(dataFullX), len(dataFullY)

(10011, 10011)

In [ ]:
maxWords = 5000
tokenizer = Tokenizer(num_words=maxWords, filters=string.punctuation + '!"#$%&()*+,-./:;<=>?@[\]^_`{|}~', lower=True)

In [ ]:
tokenizer.fit_on_texts(map(lambda x: " ".join(x), dataFullX))
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

Found 108598 unique tokens.


In [ ]:
X = tokenizer.texts_to_sequences(map(lambda x: " ".join(x), dataFullX))
X = pad_sequences(X, maxlen=sentMaxLen)
X.shape

(10011, 300)

In [ ]:
X[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,  284, 1545,    4, 1311,  488,  573,  156,  207,    4,
        783,  127,  823,  386,   59,    8,  134,  335,  429, 3788,   59,
        157,  318,   30,  310,  556,  291,  552,   59,  162,  989, 1946,
       1831, 1363,  541, 1122, 4211, 2270,   75, 3687,   59,  961, 4171,
         47,   43, 2646,  460,   22,   55,   49,   

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, dataFullY, test_size = 0.1, shuffle=True)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

(9009, 300) (9009, 4)
(1002, 300) (1002, 4)


In [ ]:
X_train[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,   

In [ ]:
embDim = 128

In [ ]:
model = Sequential([
                    Embedding(maxWords, embDim, input_length=X.shape[1]),
                    SpatialDropout1D(0.2),
                    LSTM(64, kernel_regularizer=l1_l2(0.0005)),
                    Dropout(0.2),
                    Dense(Y_test.shape[1], activation='softmax')
])
model.compile(loss='categorical_crossentropy', optimizer=Adam(0.001), metrics=['accuracy'])

In [ ]:
model.summary()


Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 300, 128)          640000    
_________________________________________________________________
spatial_dropout1d_2 (Spatial (None, 300, 128)          0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 64)                49408     
_________________________________________________________________
dropout_2 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 4)                 260       
Total params: 689,668
Trainable params: 689,668
Non-trainable params: 0
_________________________________________________________________


In [ ]:
epochs = 25
batch_size = 512

In [ ]:
def scheduler(epoch, lr):
  if epoch < 10:
    return lr
  else:
    return lr / (2.7)
history = model.fit(X_train,
                    Y_train,
                    epochs=epochs,
                    batch_size=batch_size,
                    validation_split=0.1,
                    callbacks=[
                            keras.callbacks.LearningRateScheduler(scheduler)  
                    ])

Epoch 1/25
16/16 [==============================] - 4s 120ms/step - loss: 3.9114 - accuracy: 0.3066 - val_loss: 3.2797 - val_accuracy: 0.3152
Epoch 2/25
16/16 [==============================] - 1s 79ms/step - loss: 3.0843 - accuracy: 0.3462 - val_loss: 2.5480 - val_accuracy: 0.5450
Epoch 3/25
16/16 [==============================] - 1s 79ms/step - loss: 2.3645 - accuracy: 0.5784 - val_loss: 1.9107 - val_accuracy: 0.6104
Epoch 4/25
16/16 [==============================] - 1s 79ms/step - loss: 1.7616 - accuracy: 0.6649 - val_loss: 1.4477 - val_accuracy: 0.7137
Epoch 5/25
16/16 [==============================] - 1s 80ms/step - loss: 1.2790 - accuracy: 0.7699 - val_loss: 1.1596 - val_accuracy: 0.7336
Epoch 6/25
16/16 [==============================] - 1s 80ms/step - loss: 0.9914 - accuracy: 0.8019 - val_loss: 1.0429 - val_accuracy: 0.7458
Epoch 7/25
16/16 [==============================] - 1s 79ms/step - loss: 0.8152 - accuracy: 0.8232 - val_loss: 0.9015 - val_accuracy: 0.7558
Epoch 8/25
1

In [ ]:
model.evaluate(X_test, Y_test)

32/32 [==============================] - 0s 9ms/step - loss: 0.7378 - accuracy: 0.7814


[0.7378488779067993, 0.78143709897995]

In [ ]:
model.metrics_names

['loss', 'accuracy']

In [ ]:
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
testDf = pd.read_csv('/content/drive/MyDrive/aiijc_sber/test.csv')

In [ ]:
testDf.shape

(514, 2)

In [ ]:
Xtest = tokenizer.texts_to_sequences(map(lambda x: str(x['task']), testDf.iloc))
Xtest = pad_sequences(Xtest, maxlen=sentMaxLen)

In [ ]:
predictions = model.predict(Xtest)
predictions

array([[0.05298402, 0.5866494 , 0.22541152, 0.13495508],
       [0.02405234, 0.8612694 , 0.07303725, 0.04164104],
       [0.07443149, 0.27244335, 0.52692974, 0.12619546],
       ...,
       [0.02555496, 0.8334557 , 0.07907335, 0.06191602],
       [0.07443387, 0.6249237 , 0.19366634, 0.10697617],
       [0.05720105, 0.73691344, 0.12170562, 0.08417981]], dtype=float32)

In [ ]:
predictions = list(map(lambda x: categories[int(x)], np.argmax(predictions, axis=-1)))
predictions[:5]

['музыка', 'музыка', 'спорт', 'музыка', 'музыка']

In [ ]:
from rake_nltk import Rake

def getKeyWords(textSeries: pd.Series, threshold=4, minLen=1, maxLen=2):
    r = Rake(
        stopwords=stopWordsRu,
        punctuations=punctuation,
        min_length=1, max_length=2
    )
    results = []
    for i in range(textSeries.shape[0]):
      r.extract_keywords_from_text(testDf['task'].iloc[i])
      rowResult = r.get_ranked_phrases_with_scores()
      selected = 1
      while selected < len(rowResult):
        if rowResult[selected][0] < threshold:
          break
        selected += 1
      results.append(rowResult[:selected])
    return list(map(lambda task: list(map(lambda keyword: keyword[1], task)), results))

In [ ]:
keyWords = getKeyWords(testDf['task'])

In [ ]:
keyWordsReady = list(map( lambda x: ";".join(x), keyWords))

In [ ]:
sample = pd.read_csv('/content/drive/MyDrive/aiijc_sber/sample_submission.csv')


In [ ]:
sample['keywords'] = keyWordsReady

In [ ]:
sample['category'] = predictions

In [ ]:
sample.to_csv('submit.csv', index=False)

In [ ]:
sample['category'].value_counts()

музыка        434
спорт          56
литература     22
животные        2
Name: category, dtype: int64

In [ ]:
sample

,id,category,keywords
0,0,музыка,настольному хоккею;вариантов выбора
1,1,музыка,собак поместится;реши задачу
2,2,спорт,течение года;сдавали зачет;реши задачу;равно и...
3,3,музыка,суммы затрат;продукты питания;конце месяца;дом...
4,4,музыка,этими объектами;увлекающихся футболом;либо пон...
...,...,...,...
509,509,музыка,следующие вопросы;проанализируй диаграмму
510,510,музыка,сборке тренажеров;выполнить задание;6 дней;4 д...
511,511,музыка,7 дней;1 день
512,512,музыка,ухватил пернатого;улетающую птицу;спрыгнул вни...
